## 1. 模型简介
[PP-YOLO](https://arxiv.org/abs/2007.12099)是PaddleDetection优化和改进的YOLOv3的模型，其精度(COCO数据集mAP)和推理速度均优于[YOLOv4](https://arxiv.org/abs/2004.10934)模型。关于PP-YOLO的更多细节可以参考[官方文档](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/ppyolo/README_cn.md)。

## 2. 模型效果
PP-YOLO在[COCO](http://cocodataset.org) test-dev2017数据集上精度达到45.9%，在单卡V100上FP32推理速度为72.9 FPS, V100上开启TensorRT下FP16推理速度为155.6 FPS。

<div align="center">
  <img src="https://raw.githubusercontent.com/PaddlePaddle/PaddleDetection/release/2.5/docs/images/ppyolo_map_fps.png" width=500 />
</div>

## 3. 模型如何使用
首先克隆PaddleDetection，并将数据集存放在`dataset/coco/`目录下面

In [ ]:
%cd ~/work
!git clone https://gitee.com/paddlepaddle/PaddleDetection
%cd PaddleDetection
!pip install -r requirements.txt

### 3.1 训练
使用8GPU通过如下命令一键式启动训练(以下命令均默认在PaddleDetection根目录运行), 通过`--eval`参数开启训练中交替评估。

In [ ]:
!python -m paddle.distributed.launch --log_dir=./ppyolo_dygraph/ --gpus 0,1,2,3,4,5,6,7 tools/train.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml

### 3.2 推理部署
PP-YOLO模型部署及推理benchmark需要通过`tools/export_model.py`导出模型后使用Paddle预测库进行部署和推理，可通过如下命令一键式启动。

In [ ]:
# 导出模型，默认存储于output/ppyolo目录
!python tools/export_model.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyolo_r50vd_dcn_1x_coco.pdparams

# 预测库推理
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyolo_r50vd_dcn_1x_coco --image_file=demo/000000014439_640x640.jpg --device=GPU

## 4. 模型原理
PP-YOLO的整体结构图如下图所示:
<div align="center">
  <img src="https://paddledet.bj.bcebos.com/modelcenter/images/PP-YOLO-Arch.png" width=500 />
</div>

PP-YOLO从如下方面优化和提升YOLOv3模型的精度和速度：

- 更优的骨干网络: ResNet50vd-DCN
- 更大的训练batch size: 8 GPUs，每GPU batch_size=24，对应调整学习率和迭代轮数
- [Drop Block](https://arxiv.org/abs/1810.12890)
- [Exponential Moving Average](https://www.investopedia.com/terms/e/ema.asp)
- [IoU Loss](https://arxiv.org/pdf/1902.09630.pdf)
- [Grid Sensitive](https://arxiv.org/abs/2004.10934)
- [Matrix NMS](https://arxiv.org/pdf/2003.10152.pdf)
- [CoordConv](https://arxiv.org/abs/1807.03247)
- [Spatial Pyramid Pooling](https://arxiv.org/abs/1406.4729)

更多细节可以参考我们的技术报告：https://arxiv.org/abs/2007.12099

## 5. 注意事项
**所有的命令默认运行在AI Studio的`jupyter`上, 如果运行在终端上，去掉命令开头的符号%或!**

## 6. 相关论文及引用信息
```
@misc{long2020ppyolo,
title={PP-YOLO: An Effective and Efficient Implementation of Object Detector},
author={Xiang Long and Kaipeng Deng and Guanzhong Wang and Yang Zhang and Qingqing Dang and Yuan Gao and Hui Shen and Jianguo Ren and Shumin Han and Errui Ding and Shilei Wen},
year={2020},
eprint={2007.12099},
archivePrefix={arXiv},
primaryClass={cs.CV}
}
```